# Лабораторная работа №5

# Разработка единого шаблона предварительной обработки данных

## Подключение библиотек

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Загрузка данных и разделение на матрицу признаков и зависимую переменную

За основу взяты сгенерированные данные, включающие в себя:
- производителя авто
- год выпуска модели (есть пропуски)
- страну эксплуатации (есть пропуски)
- километраж (в милях) (есть пропуски)
- флаг о нужде в починке

In [46]:
dataset = pd.read_csv('MOCK_DATA.csv')
dataset.head()

,Car Make,Country,Year,Mileage,Needs Repair
0,Lotus,Nigeria,2005.0,491887.0,False
1,Pontiac,Central African Republic,1971.0,490908.0,True
2,Hummer,Ireland,1997.0,NaN,False
3,Suzuki,Sweden,2002.0,136542.0,False
4,Mitsubishi,China,1999.0,NaN,True


In [47]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
print("Матрица признаков"); print(X[:5])
print("Зависимая переменная"); print(y[:5])

Матрица признаков
[['Lotus' 'Nigeria' 2005.0 491887.0]
 ['Pontiac' 'Central African Republic' 1971.0 490908.0]
 ['Hummer' 'Ireland' 1997.0 nan]
 ['Suzuki' 'Sweden' 2002.0 136542.0]
 ['Mitsubishi' 'China' 1999.0 nan]]
Зависимая переменная
[False  True False False  True]


## Обработка пропущенных значений

In [48]:
# Новая версия класса-трансформера, без устаревшего Imputer
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X[:, 2:4])
X_without_nan = X.copy()
X_without_nan[:, 2:4] = imputer.transform(X[:, 2:4])
X_without_nan

array([['Lotus', 'Nigeria', 2005.0, 491887.0],
       ['Pontiac', 'Central African Republic', 1971.0, 490908.0],
       ['Hummer', 'Ireland', 1997.0, 260451.41992882563],
       ...,
       ['Cadillac', 'Venezuela', 1995.0, 324987.0],
       ['Ford', 'China', 2008.0, 44461.0],
       ['Audi', 'Japan', 2000.0, 58580.0]], dtype=object)

## Обработка категориальных данных

### Замена категории кодом (LabelEncoder)

In [49]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
print("Зависимая переменная до обработки")
print(y[:10])
y = labelencoder_y.fit_transform(y)
print("Зависимая переменная после обработки")
print(y[:10])

Зависимая переменная до обработки
[False  True False False  True  True False False  True False]
Зависимая переменная после обработки
[0 1 0 0 1 1 0 0 1 0]


### Применение OneHotEncoder

In [50]:
# создаем копию "грязного" объекта: спропусками и некодированными категориями
X_dirty = X.copy()
X_dirty

array([['Lotus', 'Nigeria', 2005.0, 491887.0],
       ['Pontiac', 'Central African Republic', 1971.0, 490908.0],
       ['Hummer', 'Ireland', 1997.0, nan],
       ...,
       ['Cadillac', 'Venezuela', 1995.0, 324987.0],
       ['Ford', 'China', 2008.0, 44461.0],
       ['Audi', 'Japan', 2000.0, 58580.0]], dtype=object)

In [51]:
# Современный метод трансформации признаков
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# создаем список трансформеров
transformers = [
    ('onehot', OneHotEncoder(), [0, 1]),
    ('imp', SimpleImputer(), [2, 3])
]

# Создаем объект ColumnTransformer и передаем ему список трансформеров
ct = ColumnTransformer(transformers)

# Выполняем трансформацию признаков
X_transformed = ct.fit_transform(X_dirty)
print(X_transformed.shape)
X_transformed  # use small data to see results

(1000, 178)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4000 stored elements and shape (1000, 178)>

In [52]:
# Можно преобразовать полученный многомерный массив обратно в Dataframe (но там получается 178 столбцов - слишком долго обрабатывается)
X_data = pd.DataFrame(
    X_transformed)
X_data.shape

(1000, 1)

## Разделение выборки на train-test

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_without_nan, y, test_size=0.2, random_state=42
)

In [54]:
X_train

array([['Rolls-Royce', 'Portugal', 2011.0, 200050.0],
       ['Buick', 'Philippines', 1993.0, 44589.0],
       ['Ford', 'Luxembourg', 1989.0, 330695.0],
       ...,
       ['Mercury', 'United States', 1998.0, 329547.0],
       ['GMC', 'Nigeria', 1997.0, 490362.0],
       ['Lincoln', 'China', 1987.0, 435076.0]], dtype=object)

In [55]:
y_train

array([0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,

In [56]:
X_test

array([['Chevrolet', 'Indonesia', 2000.0, 116114.0],
       ['Ford', 'China', 1997.0, 459654.0],
       ['Chevrolet', 'Nigeria', 1993.0, 353215.0],
       ['Buick', 'Philippines', 2002.0, 100957.0],
       ['Mercedes-Benz', 'Indonesia', 2001.0, 281876.0],
       ['Daewoo', 'Cuba', 1999.0, 205654.0],
       ['Chevrolet', 'Bulgaria', 1995.0, 181783.0],
       ['Porsche', 'China', 1999.0, 205943.0],
       ['Plymouth', 'Indonesia', 2000.0, 497736.0],
       ['Lexus', 'Poland', 2011.0, 200194.0],
       ['Dodge', 'Niger', 1993.0, 356073.0],
       ['Ford', 'Indonesia', 2008.0, 357588.0],
       ['Mazda', 'Canada', 1992.0, 359467.0],
       ['Mitsubishi', 'Indonesia', 2000.2080173347779,
        260451.41992882563],
       ['Acura', 'France', 1998.0, 165513.0],
       ['Chrysler', 'China', 2002.0, 245611.0],
       ['GMC', 'France', 1996.0, 67763.0],
       ['Audi', 'Japan', 2000.2080173347779, 496463.0],
       ['Ford', 'China', 2012.0, 260451.41992882563],
       ['Ford', 'Poland', 2008.0

In [57]:
y_test

array([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0])